In [65]:
from smolagents import CodeAgent, tool, InferenceClientModel, load_tool, DuckDuckGoSearchTool, FinalAnswerTool
import pytz
import datetime
import requests

In [66]:
system_prompt = """You are a helpful AI assistant that can use tools.
You must think step by step and always use the tools when needed.
After you have the answer, use final_answer to return it."""

In [67]:
@tool
def get_current_time_in_timezone(timezone: str) -> str:
    """A tool that fetches the current local time in a specified timezone.
    Args:
        timezone: A string representing a valid timezone (e.g., 'America/New_York').
    """
    try:
        # Create timezone object
        tz = pytz.timezone(timezone)
        # Get current time in that timezone
        local_time = datetime.datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
        return f"The current local time in {timezone} is: {local_time}"
    except Exception as e:
        return f"Error fetching time for timezone '{timezone}': {str(e)}"

In [73]:
final_answer = FinalAnswerTool()
model = InferenceClientModel(
    token = "hf_token",
    max_tokens=2096,
    temperature=0.5,
    model_id='meta-llama/Llama-4-Scout-17B-16E-Instruct'
)

In [69]:

@tool
def image_gen(prompt: str) -> str:
    """Generate an image from text using a reliable model.
    
    Args:
        prompt: The text description of the image to generate
    """
    try:
        from huggingface_hub import InferenceClient
        client = InferenceClient(token="hf_token") 
        image = client.text_to_image(prompt, model="black-forest-labs/FLUX.1-schnell")
        image.save("generated_image.png")
        return f"✅ Image generated and saved for: '{prompt}'"
    except Exception as e:
        return f"❌ Image generation failed: {str(e)}"

In [70]:
agent = CodeAgent(
    model=model,
    tools=[final_answer, get_current_time_in_timezone, image_gen, DuckDuckGoSearchTool()],
    max_steps=6,
    verbosity_level=1,
    # Remove: grammar, planning_interval, name, description
)


In [54]:
agent.run("What time is it in Tokyo?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What time is it in Tokyo?                                                                                       │
│                                                                                                                 │
╰─ InferenceClientModel - meta-llama/Llama-4-Scout-17B-16E-Instruct ──────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  tokyo_time = get_current_time_in_timezone(timezone='Asia/Tokyo')                                                 
  print(tokyo_time)                                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The current local time in Asia/Tokyo is: 2025-11-03 23:17:40

Out: None

[Step 1: Duration 1.66 seconds| Input tokens: 2,104 | Output tokens: 84]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The current time in Tokyo is 2025-11-03 23:17:40")                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: The current time in Tokyo is 2025-11-03 23:17:40

[Step 2: Duration 0.75 seconds| Input tokens: 4,398 | Output tokens: 165]

'The current time in Tokyo is 2025-11-03 23:17:40'

In [64]:
agent.run("what is the current time in london and generate an image regarding that subject like something that is found only in london or the uk")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ what is the current time in london and generate an image regarding that subject like something that is found    │
│ only in london or the uk                                                                                        │
│                                                                                                                 │
╰─ InferenceClientModel - meta-llama/Llama-4-Scout-17B-16E-Instruct ──────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Get the current time in London                                                                                 
  current_time_london = get_current_time_in_timezone(timezone="Europe/London")                                     
  print(current_time_london)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The current local time in Europe/London is: 2025-11-03 14:27:46

Out: None

[Step 1: Duration 2.22 seconds| Input tokens: 2,120 | Output tokens: 261]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Generate an image of a London bus                                                                              
  image_of_london_bus = image_gen(prompt="a classic red London bus")                                               
  final_answer(image_of_london_bus)                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: ✅ Image generated and saved for: 'a classic red London bus'

[Step 2: Duration 3.30 seconds| Input tokens: 4,429 | Output tokens: 411]

"✅ Image generated and saved for: 'a classic red London bus'"

In [71]:
agent.run("tell me abt some countries that are 12 hrs before india describe if they are culturally rich as india. which country would you rate most rich?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ tell me abt some countries that are 12 hrs before india describe if they are culturally rich as india. which    │
│ country would you rate most rich?                                                                               │
│                                                                                                                 │
╰─ InferenceClientModel - meta-llama/Llama-4-Scout-17B-16E-Instruct ──────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # List of timezones around the world                                                                             
  timezones = {                                                                                                    
      "America/New_York": -5,  # Eastern Standard Time                                                             
      "America/Chicago": -6,  # Central Standard Time                                                              
      "America/Denver": -7,    # Mountain Standard Time                                                            
      "America/Los_Angeles": -8,  # Pacific Standard Time                                                          
      "Europe/London": 0,      # Greenwich Mean Time                                                               
      "Europe/Paris": 1,       # Central European Time                                                             
      "Asia/Kolkata": 5.5      # India Standard Time                                                               
  }                                                                                                                
                                                                                                                   
  # Countries 12 hours behind India (+5:30) would be around -6:30 or +17:30                                        
  # Let's consider -7 and +17 timezones for simplicity                                                             
                                                                                                                   
  countries_behind_india = {                                                                                       
      "USA (Mountain Time)": -7,                                                                                   
      "USA (Pacific Time)": -8,                                                                                    
      "Canada (Pacific Time)": -8,                                                                                 
      "Mexico": -7,                                                                                                
  }                                                                                                                
                                                                                                                   
  print("Countries/Territories 12 hours behind India (approx):")                                                   
  for country, offset in countries_behind_india.items():                                                           
      print(f"{country}: {offset}")                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Countries/Territories 12 hours behind India (approx):
USA (Mountain Time): -7
USA (Pacific Time): -8
Canada (Pacific Time): -8
Mexico: -7

Out: None

[Step 1: Duration 3.93 seconds| Input tokens: 2,123 | Output tokens: 997]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Mexico")                                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Mexico

[Step 4: Duration 0.90 seconds| Input tokens: 16,933 | Output tokens: 1,744]

'Mexico'

In [72]:
agent.run("I'm curious about time zones make a presentation genrated as an image explaining time zones with real time zones? Explain it in a way you wid to a six yr old")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ I'm curious about time zones make a presentation genrated as an image explaining time zones with real time      │
│ zones? Explain it in a way you wid to a six yr old                                                              │
│                                                                                                                 │
╰─ InferenceClientModel - meta-llama/Llama-4-Scout-17B-16E-Instruct ──────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  prompt = "A colorful illustration showing a child how time zones work, with clocks showing different times in    
  different zones like New York, London, and Tokyo."                                                               
  image_url = image_gen(prompt)                                                                                    
  print(image_url)                                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
✅ Image generated and saved for: 'A colorful illustration showing a child how time zones work, with clocks showing
different times in different zones like New York, London, and Tokyo.'

Out: None

[Step 1: Duration 3.67 seconds| Input tokens: 2,126 | Output tokens: 171]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("A colorful illustration showing a child how time zones work, with clocks showing different times   
  in different zones like New York, London, and Tokyo. The image has been generated and saved.")                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: A colorful illustration showing a child how time zones work, with clocks showing different times in 
different zones like New York, London, and Tokyo. The image has been generated and saved.

[Step 2: Duration 1.71 seconds| Input tokens: 4,559 | Output tokens: 524]

'A colorful illustration showing a child how time zones work, with clocks showing different times in different zones like New York, London, and Tokyo. The image has been generated and saved.'